In [14]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as plt
import torch as pt
import csv
import pickle #to save notebook at sessions

#set path for pickles to be saved in
pickle_path = '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/python pickles/'

# What is going on (on raw counts)
1. Import glycogene csv of human glycogenes converted to mice homologs (because our data is from mice tumor models. Human glycogenes are from glyco.me (Glycopacity). 
2. Load barcodes, features, and matrix of LN and TILs and convert to dataframe
3. Import transformed identity csv with info on which cells are associated with which cell type
4. Filter cells from original dataframe to produce smaller dataframe of just TILs in LN/TIL population from transformed identity csv
5. Filter genes from filtered dataframe (from 4) to only include glycogenes from (1)
6. Convert filtered dataframe back into a sparse matrix for export into .mtx file

## 1. Import human to mouse glycogene csv.
Converted human glycogenes to their mouse homologs via https://www.informatics.jax.org/batch/summary

In [2]:
glyco_fp = '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/human to mouse glycogene converted.csv'
glycogene_df = pd.read_csv(glyco_fp)
#make mouse glycogene list
m_glyco = glycogene_df[['Mouse']]
mouse_glycogenes = list(m_glyco.dropna(axis=0)['Mouse']) #removed 3 genes that didn't have mice homologs

#export just the mouse glycogenes as csv
m_glyco.to_csv('Mouse glycogenes.csv')

In [4]:
len(mouse_glycogenes)

264

## 2. Load barcodes, features, and matrix of LN and TILs and convert to dataframe


In [5]:
## TAKES LONG TIME TO RUN!! ## 
# Load the LN SUGAR-seq matrix from file
sparse_LN = sp.io.mmread(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/LN data/matrix.mtx')
# Convert to CSR format
sparse_LN = sp.sparse.csr_matrix(sparse_LN)

In [6]:
## TAKES LONG TIME TO RUN!! ## 
# Load the TIL SUGAR-seq matrix from file
sparse_TIL = sp.io.mmread(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/matrix.mtx')
# Convert to CSR format
sparse_TIL = sp.sparse.csr_matrix(sparse_TIL)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/matrix.mtx'

In [ ]:
# dimension check of sparse matrix
print('LN matrix dimesions are', np.shape(sparse_LN)) #31059 = rows , 21343=columns
print('TIL matrix dimesions are', np.shape(sparse_TIL))

# convert sparse matrices to array to put into dataframe
LN_matrix = sparse_LN.toarray()
TIL_matrix = sparse_TIL.toarray()

### Set up data frame for TIL

In [ ]:
### TIL data ##  
# barcodes = row names
barcodes = pd.read_csv(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/barcodes copy.tsv', 
    header=None)
barcodes = barcodes.rename(columns={0: 'barcode'})
TIL_barcodes = list(barcodes['barcode']) # of columns =  21343

# features = column names
feats = pd.read_csv(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/features.tsv', 
    header = None)
with open('/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/features.tsv', 
          'r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    feat_ls = []
    for row in reader:
        feat_ls.append(row)
TIL_features = [i[1] for i in feat_ls] # of rows = 31059

# Convert the dense matrix to a pandas DataFrame
TIL_df = pd.DataFrame(data=TIL_matrix, index=TIL_features, columns=TIL_barcodes)

# Remove rows for HTO1, HTO2, PD1, and TIM3 (ADT and HTO assays)
TIL_df_new = TIL_df.drop(['HTO1', 'HTO2','PD1', 'TIM3'])

### set up dataframe for LN

In [ ]:
## LN data ## 
# barcodes = row names
barcodes = pd.read_csv(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/LN data/barcodes copy.tsv', 
    header=None)
barcodes = barcodes.rename(columns={0: 'barcode'})
LN_barcodes = list(barcodes['barcode']) # of columns =  21343

# features = column names
feats = pd.read_csv(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/LN data/features.tsv', header = None)
with open('/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/LN data/features.tsv', 
          'r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    feat_ls = []
    for row in reader:
        feat_ls.append(row)
LN_features = [i[1] for i in feat_ls] # of rows = 31059

# Convert the dense matrix to a pandas DataFrame with columns as each barcode and rows as each gene
LN_df = pd.DataFrame(data=LN_matrix, index=LN_features, columns=LN_barcodes)

# Remove rows for HTO1, HTO2, PD1, and TIM3 (ADT and HTO assays)
LN_df_new = LN_df.drop(['hashtag1', 'hashtag2','hashtag3', 'PD1_hash', 'TIM3_hash'])

In [ ]:
with open(pickle_path + 'TIL df set up.pkl', 'wb') as f:
    pickle.dump(TIL_df_new, f)
    
with open(pickle_path + 'LN df set up.pkl', 'wb') as f:
    pickle.dump(LN_df_new, f)

f.close()

## 3. Add cell type info from csv exported from ProjecTIL cell type annotation

In [7]:
with open(pickle_path + 'TIL df set up.pkl', 'rb') as f:
    TIL_df_new = pickle.load(f)

with open(pickle_path + 'LN df set up.pkl', 'rb') as f:
    LN_df_new = pickle.load(f)

In [8]:
path_TIL='/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/TIL output/ProjecTIL analysis/TIL_transformed_identity.csv'
path_LN='/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/LN output/ProjecTIL analysis/LN_transformed_identity.csv'

# import ProjecTIL analysis output to add cell type annotation to raw glycogene data frame 
TIL_transformed_ident = pd.read_csv(path_TIL)
LN_transformed_ident = pd.read_csv(path_LN)

#rename column of ProjecTIL df for easy identification of barcodes
TIL_transformed_ident = TIL_transformed_ident.rename(columns={'Unnamed: 0':'Barcodes'}) 
LN_transformed_ident = LN_transformed_ident.rename(columns={'Unnamed: 0':'Barcodes'}) 

#Transpose matrix to match final TIL/LN matrix format
TIL_transformed_ident = TIL_transformed_ident.transpose()
LN_transformed_ident = LN_transformed_ident.transpose()

In [10]:
#RUn only once, else it keeps replacing index with top row. if you run too much, rerun previous cell first!! 
'''
make the column names the barcodes, which is how the glycogene 
dataframe (the one we want to combine this one with) is formatted as, to concatenate
'''
TIL_transformed_ident.set_axis(TIL_transformed_ident.iloc[0], axis=1, inplace=True)
TIL_transformed_ident = TIL_transformed_ident[1:] 

LN_transformed_ident.set_axis(LN_transformed_ident.iloc[0], axis=1, inplace=True)
LN_transformed_ident = LN_transformed_ident[1:]

In [ ]:
#save as binarized pickle file 
with open(pickle_path + 'TIL_transformed_ident.pkl', 'wb') as f:
    pickle.dump(TIL_transformed_ident, f)
    
with open(pickle_path + 'LN_transformed_ident.pkl', 'wb') as f:
    pickle.dump(LN_transformed_ident, f)

f.close()

### Remove non-TILs from dataframe

In [12]:
# # filter original matrix by barcodes of cells identified as TILs via ProjecTILs #

# extract list of barcodes that have a T-cell annotation
TIL_barcodes = list(TIL_transformed_ident.columns)
LN_barcodes = list(LN_transformed_ident.columns)


filtered_TIL_df = TIL_df_new[TIL_barcodes]
filtered_LN_df = LN_df_new[LN_barcodes]

print('Original dataframe dimensions are ', TIL_df_new.shape, 
      'and the filtered dataframe dimensions are', filtered_TIL_df.shape)
print('Original dataframe dimensions are ', LN_df_new.shape, 
      'and the filtered dataframe dimensions are', filtered_LN_df.shape)

Original dataframe dimensions are  (31054, 19912) and the filtered dataframe dimensions are (31054, 19645)
Original dataframe dimensions are  (31054, 21343) and the filtered dataframe dimensions are (31054, 21296)


In [ ]:
#save as binarized pickle file 
with open(pickle_path + 'filtered_TIL_df.pkl', 'wb') as f:
    pickle.dump(filtered_TIL_df, f)
    
with open(pickle_path + 'filtered_LN_df.pkl', 'wb') as f:
    pickle.dump(filtered_LN_df, f)

f.close()

In [ ]:
##TAKES LONG TIME TO RUN##
'''
Add cell type information via concatenation
Note: the L-Pha information from this csv is normalized, and since we're using raw data we'll use the raw biotin info
'''
TIL_type = TIL_transformed_ident.loc[['Type']]
LN_type = LN_transformed_ident.loc[['Type']]

#makes new dataframe that adds T-cell type annotations to raw gene expression dataframe 
TIL_type_df = pd.concat([filtered_TIL_df, TIL_type])
LN_type_df = pd.concat([filtered_LN_df, LN_type])

In [ ]:
with open(pickle_path + 'TIL_type_df.pkl', 'wb') as f:
    pickle.dump(TIL_type_df, f)
    
with open(pickle_path + 'LN_type_df.pkl', 'wb') as f:
    pickle.dump(LN_type_df, f)

f.close()

## 4. Filter matrix to only include glycogenes

In [6]:
with open(pickle_path + 'TIL_type_df.pkl', 'rb') as f:
    TIL_type_df = pickle.load(f)

with open(pickle_path + 'LN_type_df.pkl', 'rb') as f:
    LN_type_df = pickle.load(f)

In [8]:
# filter the original matrix by the mouse glycogenes but keep biotin and type info
print('Total number of glycogenes looked for:', len(mouse_glycogenes))

#Look for glycogenes within TIL_type_df
TILglycogenes_found = [i for i in mouse_glycogenes if i in TIL_type_df.index]
TILglycogenes_notfound = [i for i in mouse_glycogenes if i not in TIL_type_df.index]

glycoTIL_df = TIL_type_df.loc[TILglycogenes_found + ['Biotin', 'Type']] #filter by glycogenes while keeping type&biotin
print('Number of glycogenes found in TILs:', len(glycoTIL_df)-2) #-2 accounts for extra type and biotin rows


#Look for glycogenes within LN_type_df
LNglycogenes_found = [i for i in mouse_glycogenes if i in LN_type_df.index]
LNglycogenes_notfound = [i for i in mouse_glycogenes if i not in LN_type_df.index]

glycoLN_df = LN_type_df.loc[LNglycogenes_found + ['Biotin_hash', 'Type']] #filter by glycogenes while keeping type&biotin
print('Number of glycogenes found in LNs:', len(glycoLN_df) - 2)

Total number of glycogenes looked for: 264
Number of glycogenes found in TILs: 261
Number of glycogenes found in LNs: 261


In [ ]:
with open(pickle_path + 'glycoTIL_df.pkl', 'wb') as f:
    pickle.dump(glycoTIL_df, f)
    
with open(pickle_path + 'glycoLN_df.pkl', 'wb') as f:
    pickle.dump(glycoLN_df, f)

f.close()

In [ ]:
'''
Couldn't find all the genes from glycogene list in dataset, see if we're missing any homologs etc...
THe same glycogenes that weren't found in TILs were also not found in LN
'''
TILnot = [i for i in mouse_glycogenes not in glycoTIL_df.index]
TILnot

## 5. Sort dataframe by biotin values and assign 1/0 to each 

In [16]:
#load updated df from pickle
pickle_in = open(pickle_path + "glycoTIL_df.pkl","rb")
glycoTIL_df = pickle.load(pickle_in)

pickle_in = open(pickle_path + "glycoLN_df.pkl","rb")
glycoLN_df = pickle.load(pickle_in)

In [20]:
# Transpose matrix for easy sorting
glycoTIL_df = glycoTIL_df.transpose()
glycoLN_df = glycoLN_df.transpose()

#Sort the cells based on their biotin values
final_TIL_df = glycoTIL_df.sort_values(by='Biotin', ascending = False)
#final_LN_df = glycoLN_df.sort_values(by='Biotin_hash', ascending = False)

In [21]:
'''
# swap locations of type and biotin columns for easy lectin categorization
'''
### for TIL_SCT_final: ###
TIL_fin = final_TIL_df.iloc[:, :-2]
TIL_final = TIL_fin.copy()
TIL_final.loc[:, 'Type']=list(final_TIL_df['Type'])
TIL_final.loc[:, 'Biotin norm.']=list(final_TIL_df['Biotin'])
final_TIL_df = TIL_final

### for TIL_SCT_final: ###
LN_fin = final_LN_df.iloc[:, :-2]
LN_final = LN_fin.copy()
LN_final.loc[:, 'Type']=list(final_LN_df['Type'])
LN_final.loc[:, 'Biotin norm.']=list(final_LN_df['Biotin_hash'])
final_LN_df = LN_final

In [15]:
# # save pickle 
# with open(pickle_path + 'final_TIL_df.pkl', 'wb') as f:
#     pickle.dump(final_TIL_df, f)
    
# with open(pickle_path + 'final_LN_df.pkl', 'wb') as f:
#     pickle.dump(final_LN_df, f)

# f.close()

 
# ## load updated df from pickle
pickle_in = open(pickle_path +"final_TIL_df.pkl","rb")
final_TIL_df = pickle.load(pickle_in)

# pickle_in = open(pickle_path +"final_LN_df.pkl","rb")
# final_LN_df = pickle.load(pickle_in)

EOFError: Ran out of input

In [23]:
final_TIL_df

,Ahsa1,Api5,Atp6v1e1,Bcap31,Cops6,Csnk2b,Eif3i,Eif4g2,Gdi2,Hnrnpf,...,Galntl6,Galntl5,Galnt13,Galnt15,Galnt14,Dse,Dsel,Glce,Type,Biotin norm.
TACAGTGGTTCAACCA-1,0,1,1,1,0,0,1,0,0,2,...,0,0,0,0,0,0,0,0,CD8_NaiveLike,37384
ACATACGAGTGCCATT-1,0,2,0,0,0,4,2,0,4,4,...,0,0,0,0,0,0,0,0,CD8_EffectorMemory,33249
TGTCCCATCGGCTTGG-1,0,0,0,1,0,0,1,3,1,2,...,0,0,0,0,0,0,0,0,Th1,32029
ACTGTCCAGGATGGTC-1,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,CD8_EarlyActiv,31700
GAACATCTCTGCTGTC-1,3,4,2,7,3,14,10,8,10,23,...,0,0,0,0,0,0,0,1,Treg,31402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGTCAGGGTTTGACAC-1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,CD8_NaiveLike,15
ACGGGTCTCGAACTGT-1,0,1,1,1,2,0,0,0,2,4,...,0,0,0,0,0,0,0,0,CD8_EffectorMemory,13
GCTGCTTCACCAACCG-1,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,CD8_NaiveLike,11
TGGTTCCGTTGATTCG-1,0,0,0,0,0,0,0,0,1,3,...,0,0,0,0,0,0,0,0,CD8_NaiveLike,11


In [27]:
'''
Now want to add Glycosylation (PHA-L) column that assigns:
Yes (1) to top 25%, 
No (0) to bottom 25%

categorize_lectin inputs:
- data_all: a dataframe that has
    - genes in the columns headers
    - barcodes as row indices
    - a column with biotin values to use as cutoffs for top 25% and bottom 25%
- quantile_high: upper quantile threshold (75% to represent top 25% of data below which 75% of the data falls)
- quantile_low: lower quantile threshold (25% to represent bottom 25% of data below which 25% of data falls)
- ref_col_loc: index of column in data_all that is used for categorization cutoffs

outputs:
- cutoff values for high and low biotin
- list of 3 arrays representing indices of rows that meet the high biotin cutoff, 
low biotin cutoff, and combined high and low cutoff
- number of rows that meet the high category cutoff and the low category cutoff.
'''

# Function: determine PHA-L read cut-offs for binary classification FROM BOJAR LAB
def categorize_lectin(data_all, quantile_high, quantile_low, ref_col_loc):
    cutoff = np.quantile(data_all.iloc[:,ref_col_loc], [quantile_high, quantile_low], 
                         interpolation="nearest").tolist()
    print(f"Cut-off for PHA-L high: {cutoff[0]}; Cut-off for PHA-L low: {cutoff[1]}")
    
    high_indices = np.array(data_all.iloc[:,ref_col_loc]>=cutoff[0])
    low_indices = np.array(data_all.iloc[:,ref_col_loc]<cutoff[1])
    high_low_indices = np.logical_or(high_indices, low_indices)

    high_count = high_indices.sum()
    low_count = low_indices.sum()
    
    return cutoff, [high_indices, low_indices, high_low_indices], [high_count, low_count]


# Parameters for categorize lectin function
quantile_high, quantile_low = 0.75, 0.25
ref_col_index = -1 #last column of dataframes contain biotin info

###for TILs###
cutoff, indices, count = categorize_lectin(final_TIL_df, quantile_high, quantile_low, -1)

glycosorted_TIL = final_TIL_df.copy()
glycosorted_TIL.loc[indices[0], "PHA-L"] = 1 #add 1 to columns in the first list of index
glycosorted_TIL.loc[indices[1], "PHA-L"] = 0
glycosorted_TIL = glycosorted_TIL.loc[indices[2], :] #cuts out the middle 50% of the dataframe without PHA-L 1 or 0 

###for LNs###
cutoff, indices, count = categorize_lectin(final_LN_df, quantile_high, quantile_low, -1)
glycosorted_LN = final_LN_df.copy()
glycosorted_LN.loc[indices[0], "PHA-L"] = 1
glycosorted_LN.loc[indices[1], "PHA-L"] = 0
glycosorted_LN = glycosorted_LN.loc[indices[2], :] #cuts out the middle 50% of the dataframe without PHA-L 1 or 0 

/var/folders/t_/h82q6gld28zc219dv9bkpl2r0000gq/T/ipykernel_27260/2844898492.py:24: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.iloc[:,ref_col_loc], [quantile_high, quantile_low],


Cut-off for PHA-L high: 738; Cut-off for PHA-L low: 237


/var/folders/t_/h82q6gld28zc219dv9bkpl2r0000gq/T/ipykernel_27260/2844898492.py:24: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.iloc[:,ref_col_loc], [quantile_high, quantile_low],


Cut-off for PHA-L high: 276; Cut-off for PHA-L low: 127


In [45]:
# save pickle 
with open(pickle_path + 'glycosorted_TIL.pkl', 'wb') as f:
    pickle.dump(glycosorted_TIL, f)
    
with open(pickle_path + 'glycosorted_LN.pkl', 'wb') as f:
    pickle.dump(glycosorted_LN, f)

f.close()

# # open via: 
# #load updated df from pickle
# pickle_in = open(pickle_path +"glycosorted_TIL.pkl","rb")
# glycosorted_TIL = pickle.load(pickle_in)

# pickle_in = open(pickle_path +"glycosorted_LN.pkl","rb")
# glycosorted_LN = pickle.load(pickle_in)

### Make a concatenated dataframe that combines LN and TIL t-cells

In [69]:
#only run once!!
glycoTIL = glycosorted_TIL.copy()
glycoLN = glycosorted_LN.copy()

#Add info about where t-cells are from (LN or TIL) just in case as column
glycoTIL['Location'] = ['TIL' for i in range(len(glycoTIL))]
glycoLN['Location'] = ['LN' for i in range(len(glycoLN))]

#add location info to barcode identifier
glycoTIL.index = [f"{x}_TIL" for x in glycoTIL.index]
glycoLN.index = [f"{x}_LN" for x in glycoLN.index]

In [74]:
combo_raw = pd.concat([glycoTIL, glycoLN])
print('Initial dimensions of TIL:', glycoTIL.shape)
print('Initial dimensions of LN:', glycoLN.shape)
print('Combined dimensions:', combo_raw.shape)

Initial dimensions of TIL: (9824, 265)
Initial dimensions of LN: (10595, 265)
Combined dimensions: (20419, 265)


In [3]:
combo_raw

NameError: name 'combo_raw' is not defined

In [6]:
# # save pickle 
# with open(pickle_path + 'combo_raw.pkl', 'wb') as f:
#     pickle.dump(combo_raw, f)


# f.close()

# open via: 
#load updated df from pickle
pickle_in = open(pickle_path +"combo_raw.pkl","rb")
combo_raw = pickle.load(pickle_in)


In [9]:
combo_raw[combo_raw['Type'] == 'Tfh']

,Ahsa1,Api5,Atp6v1e1,Bcap31,Cops6,Csnk2b,Eif3i,Eif4g2,Gdi2,Hnrnpf,...,Galnt13,Galnt15,Galnt14,Dse,Dsel,Glce,Type,Biotin norm.,PHA-L,Location
AATCCAGCATATGCTG-1_TIL,2,0,0,0,0,2,3,1,2,5,...,0,0,0,0,0,0,Tfh,13596,1.0,TIL
GCCTCTATCTGGTTCC-1_TIL,2,1,0,12,0,1,2,7,1,8,...,0,0,0,2,0,0,Tfh,4131,1.0,TIL
GACGTTACACGCCAGT-1_TIL,4,2,1,12,9,6,16,8,3,3,...,0,0,0,1,0,0,Tfh,2879,1.0,TIL
GGACAGACAGCTGTAT-1_TIL,0,0,0,12,0,0,0,1,1,4,...,0,0,0,1,0,0,Tfh,1781,1.0,TIL
TGACTTTTCACATAGC-1_TIL,8,3,5,7,3,3,2,23,5,27,...,0,0,0,1,1,1,Tfh,1234,1.0,TIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATTGGACGTAAACACA-1_LN,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,Tfh,119,0.0,LN
CGGACGTAGTGGAGTC-1_LN,1,1,0,1,2,2,2,4,3,2,...,0,0,0,0,0,0,Tfh,112,0.0,LN
CTACATTCATTTCAGG-1_LN,0,1,0,0,2,0,0,4,1,4,...,0,0,0,0,0,0,Tfh,102,0.0,LN
TTGCGTCCAGTCAGCC-1_LN,0,1,0,0,0,1,2,1,1,5,...,0,0,0,0,0,0,Tfh,76,0.0,LN


### SPlit by subtype

In [46]:
'''
Now want to split up glycosorted dataframe into sub-dataframes for
each of the different T-cell subtypes. TO do this, this code block takes in the glycosorted
dataframe data as the input and spits out a dictionaryt containing:
- t-cell type as key 
- matrix containing raw expression data for dataframe as values

'''
# Get list of T-cell subtypes for which to make a dataframe for TILs
tcell_subtypes = glycosorted_TIL['Type'].unique()

#make dataframe names for later access
df_names = [i+'_df' for i in list(tcell_subtypes)]

# Make copy of original df just in case
split_df = glycosorted_TIL.copy()

TILtcell_dfs = {}

# Make separate dataframe containing data for each t-cell subtype
for cell_type, name in zip(tcell_subtypes, df_names):
    TILtcell_dfs[name] = split_df[split_df['Type'] == cell_type]
    
###----------------------------------------------------------------------
# Get list of T-cell subtypes for which to make a dataframe for LNs
tcell_subtypes = glycosorted_LN['Type'].unique()

#make dataframe names for later access
df_names = [i+'_df' for i in list(tcell_subtypes)]


# Make copy of original df just in case
split_df = glycosorted_LN.copy()

LNtcell_dfs = {}
# Make separate dataframe containing data for each t-cell subtype
for cell_type, name in zip(tcell_subtypes, df_names):
    LNtcell_dfs[name] = split_df[split_df['Type'] == cell_type]

In [47]:
# save pickle 
with open(pickle_path + 'TILtcell_dfs.pkl', 'wb') as f:
    pickle.dump(TILtcell_dfs, f)
    
with open(pickle_path + 'LNtcell_dfs.pkl', 'wb') as f:
    pickle.dump(LNtcell_dfs, f)

f.close()

# open via: 
# #load updated df from pickle
# pickle_in = open(pickle_path + "TILtcell_dfs.pkl","rb")
# TILtcell_dfs = pickle.load(pickle_in)

# pickle_in = open(pickle_path + "LNtcell_dfs.pkl","rb")
# LNtcell_dfs = pickle.load(pickle_in)